## Etapa 1: Importação de Bibliotecas Necessárias

In [ ]:
import pandas as pd
import gdown
import plotly.express as px
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler

## Etapa 2: Carregamento de Dados

In [2]:
# Base do nome do arquivo destino
arquivo_destino_base = "dataset_{}.csv"

# IDs dos arquivos no Google Drive
ids = {
    "dados": "1oEXsdLnaLxsnf7LURAdgAy5ZjICFuQGv",
}

# Dicionário para armazenar DataFrames
dataframes = {}

# Loop para baixar e ler cada arquivo
for key, file_id in ids.items():
    url = f"https://drive.google.com/uc?id={file_id}"
    arquivo_destino = arquivo_destino_base.format(key)

    # Baixa o arquivo usando gdown
    gdown.download(url, arquivo_destino, quiet=False)

    # Tenta ler o arquivo com pandas
    try:
        df = pd.read_csv(arquivo_destino)
        dataframes[key] = df
    except pd.errors.ParserError:
        print(f"Erro ao ler o arquivo {arquivo_destino}. Verifique o separador.")

Downloading...
From (original): https://drive.google.com/uc?id=1oEXsdLnaLxsnf7LURAdgAy5ZjICFuQGv
From (redirected): https://drive.google.com/uc?id=1oEXsdLnaLxsnf7LURAdgAy5ZjICFuQGv&confirm=t&uuid=5da54418-b58e-41fd-b4da-911a36017f32
To: /content/dataset_dados.csv
100%|██████████| 683M/683M [00:06<00:00, 108MB/s]


In [3]:
dados = pd.read_csv("/content/dataset_dados.csv", delimiter=",")
# display(dados)

##Etapa 3: Análise Exploratória dos Dados (EDA)
Nessa etapa realizamos uma análise exploratória detalhada para entender as distribuições, detectar outliers e identificar possíveis padrões nas transações fraudulentas. Para isso, foram incluídos:
- Contagem de dados nulos
- Distribuição de variáveis com histogramas ou boxplots;
- Correlações entre variáveis com heatmaps;
- Identificação de outliers com gráficos de dispersão e boxplots; e
- Contagem e análise de transações fraudulentas vs. não fraudulentas

In [4]:
# Informações iniciais do dataset
dados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 590540 entries, 0 to 590539
Columns: 394 entries, TransactionID to V339
dtypes: float64(376), int64(4), object(14)
memory usage: 1.7+ GB


In [5]:
# Análise estatística descritiva
print("Tipos de dados das features:")
display(dados.dtypes)

Tipos de dados das features:


,0
TransactionID,int64
isFraud,int64
TransactionDT,int64
TransactionAmt,float64
ProductCD,object
...,...
V335,float64
V336,float64
V337,float64
V338,float64


In [6]:
print("Descrição estatística das features numéricas:")
display(dados.describe())

Descrição estatística das features numéricas:


,TransactionID,isFraud,TransactionDT,TransactionAmt,card1,card2,card3,card5,addr1,addr2,...,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339
count,5.905400e+05,590540.000000,5.905400e+05,590540.000000,590540.000000,581607.000000,588975.000000,586281.000000,524834.000000,524834.000000,...,82351.000000,82351.000000,82351.000000,82351.000000,82351.000000,82351.00000,82351.000000,82351.000000,82351.000000,82351.000000
mean,3.282270e+06,0.034990,7.372311e+06,135.027176,9898.734658,362.555488,153.194925,199.278897,290.733794,86.800630,...,0.775874,721.741883,1375.783644,1014.622782,9.807015,59.16455,28.530903,55.352422,151.160542,100.700882
std,1.704744e+05,0.183755,4.617224e+06,239.162522,4901.170153,157.793246,11.336444,41.244453,101.741072,2.690623,...,4.727971,6217.223583,11169.275702,7955.735482,243.861391,387.62948,274.576920,668.486833,1095.034387,814.946722
min,2.987000e+06,0.000000,8.640000e+04,0.251000,1000.000000,100.000000,100.000000,100.000000,100.000000,10.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000
25%,3.134635e+06,0.000000,3.027058e+06,43.321000,6019.000000,214.000000,150.000000,166.000000,204.000000,87.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000
50%,3.282270e+06,0.000000,7.306528e+06,68.769000,9678.000000,361.000000,150.000000,226.000000,299.000000,87.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000
75%,3.429904e+06,0.000000,1.124662e+07,125.000000,14184.000000,512.000000,150.000000,226.000000,330.000000,87.000000,...,0.000000,0.000000,25.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000
max,3.577539e+06,1.000000,1.581113e+07,31937.391000,18396.000000,600.000000,231.000000,237.000000,540.000000,102.000000,...,55.000000,160000.000000,160000.000000,160000.000000,55125.000000,55125.00000,55125.000000,104060.000000,104060.000000,104060.000000


In [7]:
# Verificar e ordenar as colunas com mais valores nulos
nulos_por_coluna = dados.isnull().sum()

# Filtrar apenas as colunas com valores nulos e ordenar em ordem decrescente
colunas_com_mais_nulos = nulos_por_coluna[nulos_por_coluna > 0].sort_values(ascending=False)
pd.set_option('display.max_rows', None)  # Remove a limitação de linhas exibidas

##Etapa 4: Tratamento de Valores Ausentes
Nessa etapa estamos focados em identificar as colunas com valores ausentes e aplicar técnicas de preenchimento ou remoção, dependendo da proporção de dados ausentes. Use métodos como:
- Preenchimento com média, mediana ou moda
- Remoção de linhas/colunas com muitos valores ausentes

In [8]:
# Definir o limite para remover colunas com mais de 50% de valores nulos
limite = len(dados) * 0.5
dados_sem_muitos_nulos = dados.dropna(thresh=limite, axis=1)

# Comparação do número de colunas antes e depois da remoção
info_comparacao = pd.DataFrame({
    'Status': ['Antes da Remoção', 'Depois da Remoção'],
    'Linhas': [dados.shape[0], dados_sem_muitos_nulos.shape[0]],
    'Colunas': [dados.shape[1], dados_sem_muitos_nulos.shape[1]]
})

In [9]:
# Comparação do número de colunas antes e depois da remoção
info_comparacao = pd.DataFrame({
    'Status': ['Antes da Remoção', 'Depois da Remoção'],
    'Linhas': [dados.shape[0], dados_sem_muitos_nulos.shape[0]],
    'Colunas': [dados.shape[1], dados_sem_muitos_nulos.shape[1]]
})

# Exibir a tabela de comparação
display(info_comparacao)

,Status,Linhas,Colunas
0,Antes da Remoção,590540,394
1,Depois da Remoção,590540,220


In [10]:
# Identificar as colunas com menos de 50% de valores nulos
colunas_com_menos_nulos = dados.columns[dados.isnull().mean() < 0.5]

# Preencher os valores nulos dessas colunas com a mediana
for coluna in colunas_com_menos_nulos:
    if dados[coluna].dtype in ['float64', 'int64']:  # Aplicar apenas a colunas numéricas
        mediana = dados[coluna].median()
        dados[coluna].fillna(mediana, inplace=True)

# Verificar se ainda restam valores nulos
print("\nVerificação de valores nulos após preenchimento com a mediana:")
display(dados.isnull().sum().sort_values(ascending=False).head(10))


Verificação de valores nulos após preenchimento com a mediana:


,0
dist2,552913
D7,551623
D13,528588
D14,528353
D12,525823
D6,517353
D9,515614
D8,515614
V153,508595
V139,508595


##Etapa 5: Feature Engineering
Descrição: Realize a criação ou transformação de novas features que possam ser úteis para o modelo. Isso pode incluir:
- Transformação de variáveis categóricas em variáveis numéricas (one-hot encoding)
- Criação de variáveis temporais ou agregadas
- Seleção de variáveis mais relevantes através de métodos como feature importance ou análise de variância

### Transformação de Variáveis Categóricas (One-Hot Encoding)


In [11]:
# Verificar se existem colunas categóricas
colunas_categoricas = dados.select_dtypes(include=['object']).columns

# Aplicar One-Hot Encoding nas variáveis categóricas, se houver
if len(colunas_categoricas) > 0:
    dados = pd.get_dummies(dados, columns=colunas_categoricas, drop_first=True)
    print(f"Colunas categóricas transformadas em dummies: {colunas_categoricas}")
else:
    print("Não há colunas categóricas para transformar.")

Colunas categóricas transformadas em dummies: Index(['ProductCD', 'card4', 'card6', 'P_emaildomain', 'R_emaildomain', 'M1',
       'M2', 'M3', 'M4', 'M5', 'M6', 'M7', 'M8', 'M9'],
      dtype='object')


- Identifica colunas categóricas (se houver).
- Transforma as colunas categóricas em colunas numéricas chamadas de dummies (variáveis que assumem valores binários, 0 ou 1).

### Criação de Variáveis temporais

In [12]:
# Exemplo: Criar uma nova feature como a razão entre duas colunas
if 'V1' in dados.columns and 'V2' in dados.columns:  # Verifique se as colunas existem
    dados['Razao_V1_V2'] = dados['V1'] / (dados['V2'] + 1e-9)  # Evitar divisão por zero
    print("Nova feature 'Razao_V1_V2' criada.")


Nova feature 'Razao_V1_V2' criada.


- Verifica se as colunas V1 e V2 existem no dataset.
- Cria uma nova variável (Razao_V1_V2) que é a divisão dos valores de V1 pelos valores de V2.

Criar novas variáveis a partir das existentes pode fornecer mais informações ao modelo e ajudar a detectar padrões.


## Etapa 6: Normalização/Padronização
Descrição: Aplique normalização ou padronização às features numéricas, o que é essencial para o desempenho do LSTM. Use métodos como:
- Min-Max Scaling
- Z-score (padronização)

In [13]:
# Escolher a técnica de normalização/padronização
scaler = MinMaxScaler()  # ou use StandardScaler() para padronização

# Aplicar a normalização nas colunas numéricas
colunas_numericas = dados.select_dtypes(include=['float64', 'int64']).columns
dados[colunas_numericas] = scaler.fit_transform(dados[colunas_numericas])

print("Normalização aplicada nas colunas numéricas.")

Normalização aplicada nas colunas numéricas.


- Seleciona todas as colunas numéricas do dataset.
- Normaliza os valores dessas colunas para que todos os valores fiquem entre 0 e 1, utilizando o Min-Max Scaling.

In [ ]:
# Supondo que você tenha carregado o 'dados_sem_muitos_nulos' antes
# Escolher a técnica de normalização (Min-Max Scaling)
scaler = MinMaxScaler()

# Selecionar todas as colunas numéricas que foram limpas (sem nulos)
colunas_numericas = dados_sem_muitos_nulos.select_dtypes(include=['float64', 'int64']).columns

# Aplicar a normalização nas colunas numéricas
dados_sem_muitos_nulos[colunas_numericas] = scaler.fit_transform(dados_sem_muitos_nulos[colunas_numericas])

# Salvando o dataset processado até a etapa 6
dados_sem_muitos_nulos.to_csv('/content/drive/MyDrive/Colab Notebooks/dados_sem_muitos_nulos_normalizados.csv', index=False)


- **Seleção das colunas numéricas limpas**: O código identifica as colunas numéricas que foram limpas na Etapa 4 (sem valores nulos).
- **Aplicação do Min-Max Scaling**: A normalização é aplicada a essas colunas, ajustando os valores para uma faixa entre 0 e 1.
- **Exibição dos dados normalizados**: Após a normalização, as primeiras linhas das colunas numéricas são exibidas para visualização.

## Etapa 7: Criação de Sequências Temporais
Descrição: Prepare os dados no formato apropriado para o LSTM, criando sequências temporais. Isso pode incluir:
- Dividir os dados em janelas temporais
- Ajustar os dados de entrada para atender ao formato (samples, timesteps, features)


In [ ]:
# Selecionando 10% do dataset
df_half = dados_sem_muitos_nulos.sample(frac=0.1, random_state=42)

# Definindo o número de timesteps e features
timesteps = 1
num_features = df_half.shape[1] - 1

# Preparar os dados
X = df_half.drop(columns=['isFraud']).values
y = df_half['isFraud'].values

# Redimensionar para o formato
X = X.reshape((X.shape[0], timesteps, num_features))

# Balanceamento com SMOTE
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X.reshape(X.shape[0], -1), y)  # reshape para aplicar SMOTE
X_resampled = X_resampled.reshape((X_resampled.shape[0], timesteps, num_features))

# Divisão entre treino e teste
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.3, random_state=42)


## Etapa 8: Definição da Arquitetura do Modelo LSTM
Descrição: Defina a estrutura da rede LSTM, escolhendo:
- Número de camadas LSTM
- Número de neurônios por camada
- Função de ativação
- Dropout (se necessário)
- Hiperparâmetros de otimização (otimizador, taxa de aprendizado, etc.)
- Função de perda e métricas


## Etapa 9: Divisão do Conjunto de Dados (Treinamento/Validação/Teste)
Descrição: Divida os dados em conjunto de treinamento, validação e teste. Certifique-se de que a divisão preserve a sequência temporal. Utilize funções como train_test_split ou validação cruzada temporal.


## Etapa 10: Treinamento do Modelo
Descrição: Treine o modelo LSTM no conjunto de treinamento, ajuste os hiperparâmetros e monitore o desempenho em termos de métricas e curvas de aprendizado. Garanta que o modelo não esteja overfitting (ex: use callbacks como EarlyStopping).

## Etapa 11: Avaliação do Modelo
Descrição: Avalie o modelo usando o conjunto de teste, aplicando métricas como:
- Precisão
- Recall
- F1-Score
- AUC-ROC
- Confusion Matrix

##Etapa 12: Análise das Curvas de Aprendizado
Descrição: Examine as curvas de perda e precisão ao longo do treinamento. Identifique possíveis sinais de overfitting ou underfitting e ajuste o modelo conforme necessário.

## Etapa 13: Ajustes Finais no Modelo
Descrição: Aplique técnicas de regularização (Dropout, L2 regularization) e ajuste de hiperparâmetros para melhorar o desempenho do modelo, se necessário. Repita o treinamento e avaliação até atingir um desempenho satisfatório.

## Etapa 14: Análise e Discussão dos Resultados
Descrição: Compare o desempenho do modelo ao longo do treinamento e teste. Discuta os principais insights obtidos com o modelo e estratégias usadas para melhorar a performance.